In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [3]:
import re
import pickle
import warnings 
warnings.filterwarnings("ignore")

In [4]:
train_df = pd.read_csv('train_tweets.csv')
train_df.drop(columns=['id'])
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
#1 Удалим @user из всех твитов с помощью паттерна "@[\w]*".

def del_sub(tweet, pattern):
    r = re.findall(pattern, tweet)
    for i in r:
        tweet = re.sub(i, '', tweet)
    return tweet

train_df["tweet"] = np.vectorize(del_sub)(train_df['tweet'], "@[\w]*")

In [6]:
#2 Изменим регистр твитов на нижний с помощью .lower()
train_df['tweet'].str.lower().head()

0      when a father is dysfunctional and is so sel...
1      thanks for #lyft credit i can't use cause th...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

In [7]:
#3 Заменим сокращения с апострофами (пример: ain't, can't) на пробел,
#  используя apostrophe_dict. Для этого необходимо сделать функцию:
#  для каждого слова в тексте проверить (for word in text.split()), если
# слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова),
# то заменить ключ на значение (полную версию слова).

apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def rep_words(tweet):
    
    for word in tweet.split():
        if word.lower() in apostrophe_dict:
            tweet = tweet.replace(word, apostrophe_dict[word.lower()])
    return tweet

train_df['tweet'] = train_df['tweet'].apply(rep_words)
train_df['tweet'].head()

0      when a father is dysfunctional and is so sel...
1      thanks for #lyft credit i cannot use cause t...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

In [8]:
#4 Заменим сокращения на их полные формы, используя short_word_dict.
#  Для этого воспользуемся функцией, используемой в предыдущем пункте.

short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

def short_words(tweet):
    
    for word in tweet.split():
        if word.lower() in short_word_dict:
            tweet = tweet.replace(word, short_word_dict[word.lower()])
    return tweet

train_df['tweet'] = train_df['tweet'].apply(short_words)
train_df['tweet'].head()

0      when a father is dysfunctional and is so sel...
1      thanks for #lyft credit i cannot use cause t...
2                                  bihday your majesty
3    #model   i love yoyou take with yoyou all the ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

In [9]:
#5 Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя
#  emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем
#  пункте.

emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

def emoticon(tweet):
    
    for word in tweet.split():
        if word.lower() in emoticon_dict:
            tweet = tweet.replace(word, emoticon_dict[word.lower()])
    return tweet

train_df['tweet'] = train_df['tweet'].apply(emoticon)
train_df['tweet'].head()

0      when a father is dysfunctional and is so sel...
1      thanks for #lyft credit i cannot use cause t...
2                                  bihday your majesty
3    #model   i love yoyou take with yoyou all the ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

In [10]:
#6 Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

def del_punct(tweet):
    for word in tweet.split():
        tweet = re.sub(r'[^\w\s]', '', tweet)
    return tweet

train_df["tweet"] = train_df["tweet"].apply(del_punct)
train_df["tweet"].head()

0      when a father is dysfunctional and is so sel...
1      thanks for lyft credit i cannot use cause th...
2                                  bihday your majesty
3    model   i love yoyou take with yoyou all the t...
4                 factsguide society now    motivation
Name: tweet, dtype: object

In [11]:
#7 Заменим спец. символы на пробелы, используя re.sub() и паттерн
#  r'[^a-zA-Z0-9]'

def del_sp_char(tweet):
    for word in tweet.split():
        tweet = re.sub(r'[^a-zA-Z0-9]', ' ', tweet)
    return tweet

train_df["tweet"] = train_df["tweet"].apply(del_sp_char)
train_df["tweet"].head()

0      when a father is dysfunctional and is so sel...
1      thanks for lyft credit i cannot use cause th...
2                                  bihday your majesty
3    model   i love yoyou take with yoyou all the t...
4                 factsguide society now    motivation
Name: tweet, dtype: object

In [12]:
#8 Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

def del_dig(tweet):
    for word in tweet.split():
        tweet = re.sub(r'[^a-zA-Z]', ' ', tweet)
    return tweet

train_df["tweet"] = train_df["tweet"].apply(del_dig)
train_df["tweet"].head()

0      when a father is dysfunctional and is so sel...
1      thanks for lyft credit i cannot use cause th...
2                                  bihday your majesty
3    model   i love yoyou take with yoyou all the t...
4                 factsguide society now    motivation
Name: tweet, dtype: object

In [13]:
#9 Удалим из текста слова длиной в 1 символ, используя 
#  ' '.join([w for w in x.split() if len(w)>1])

train_df["tweet"] = train_df["tweet"].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
train_df["tweet"].head()

0    when father is dysfunctional and is so selfish...
1    thanks for lyft credit cannot use cause they d...
2                                  bihday your majesty
3    model love yoyou take with yoyou all the time ...
4                    factsguide society now motivation
Name: tweet, dtype: object

In [14]:
#10 Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав
#   новый столбец 'tweet_token'.

train_df["tweet_token"] = train_df["tweet"].apply(word_tokenize)
train_df["tweet_token"].head()

0    [when, father, is, dysfunctional, and, is, so,...
1    [thanks, for, lyft, credit, can, not, use, cau...
2                              [bihday, your, majesty]
3    [model, love, yoyou, take, with, yoyou, all, t...
4               [factsguide, society, now, motivation]
Name: tweet_token, dtype: object

In [15]:
#11 Удалим стоп-слова из токенов, используя nltk.corpus.stopwords.
#   Создадим столбец 'tweet_token_filtered' без стоп-слов.

stop = stopwords.words('english')
train_df["tweet_token_filtered"] = train_df["tweet_token"].apply(lambda x: [item for item in x if item not in stop])
train_df["tweet_token_filtered"].head()

0    [father, dysfunctional, selfish, drags, kids, ...
1    [thanks, lyft, credit, use, cause, offer, whee...
2                                    [bihday, majesty]
3      [model, love, yoyou, take, yoyou, time, yoyour]
4                    [factsguide, society, motivation]
Name: tweet_token_filtered, dtype: object

In [16]:
#12 Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим
#   столбец 'tweet_stemmed' после применения стемминга.

ps = PorterStemmer()

def stemmer(sentence):

    stemmed_tokens = [ps.stem(token) for token in sentence]
    return stemmed_tokens

train_df["tweet_stemmed"] = train_df["tweet_token_filtered"].apply(stemmer)
train_df["tweet_stemmed"].head()

0    [father, dysfunct, selfish, drag, kid, dysfunc...
1    [thank, lyft, credit, use, caus, offer, wheelc...
2                                    [bihday, majesti]
3      [model, love, yoyou, take, yoyou, time, yoyour]
4                          [factsguid, societi, motiv]
Name: tweet_stemmed, dtype: object

In [17]:
#13 Применим лемматизацию к токенам с помощью 
#   nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' 
#   после применения лемматизации.

lemma = WordNetLemmatizer()

def lemmatize(s):
    
     s = [lemma.lemmatize(word) for word in s]
     return s

train_df["tweet_lemmatized"] = train_df["tweet_token_filtered"].apply(lambda x: lemmatize(x))
train_df["tweet_lemmatized"].head()

0    [father, dysfunctional, selfish, drag, kid, dy...
1    [thanks, lyft, credit, use, cause, offer, whee...
2                                    [bihday, majesty]
3      [model, love, yoyou, take, yoyou, time, yoyour]
4                    [factsguide, society, motivation]
Name: tweet_lemmatized, dtype: object

In [18]:
#14 Сохраним результат предобработки в pickle-файл.

train_df.head(3)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


In [19]:
with open('tweet.pkl', 'wb') as f:
    pickle.dump(train_df, f)